In [1]:
# Import der benötigten Libraries
import pandas as pd
import nltk
import numpy as np
import re
import os

pd.set_option('display.max_colwidth', 50)

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/JobCloud_Daten')
print(os.getcwd())

# Files im Directory
print(os.listdir())

C:\Users\Sven Konermann\Documents\Master\2_Semester\aF_E_Faelle\Unterlagen\JobCloud_Daten
['Anzahl_Inserate_Anforderungsdaten.pdf', 'Anzahl_Inserate_Gesamtdaten.pdf', 'ausgewertete_Anforderungen.csv', 'Daten_JobCloud001.txt', 'Daten_JobCloud002.txt', 'Daten_JobCloud003.txt', 'Fixwoerter.xlsx', 'Inserate_Anforderungsanalyse_AF_E.csv', 'Inserate_clean_Anforderungsanalyse_AF_E.csv', 'Inserate_clean_Gesamtdatenanalyse.csv', 'Inserate_clean_Titelanalyse_AF_E.csv', 'Inserate_Gesamtdatenanalyse_AF_E.csv', 'ngram_count_all_Title.csv', 'Phrasenwoerter.xlsx']


In [2]:
data = pd.read_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")
data = data[~data['Text_Anforderungen'].isnull()] # Wenn NULL-Werte, lösche diese
print(data.shape)

(132764, 16)


In [3]:
set_phrases = pd.read_excel('Phrasenwoerter.xlsx')
set_phrases.head()

,word,kategorie,kategorie_name,hauptkategorie
0,visual studio,2,Informatik Software-Kenntnisse,IT-Kenntnisse
1,big data,3,Datenmanagement,IT-Kenntnisse
2,business intelligence,3,Datenmanagement,IT-Kenntnisse
3,data science,3,Datenmanagement,IT-Kenntnisse
4,datenbank,3,Datenmanagement,IT-Kenntnisse


In [4]:
options =	{
    "Administration/HR/Consulting/CEO": ['Beratung/Unternehmensentwicklung','CEO/GeschÃ¤ftsfÃ¼hrung','Direktionsassistenz','Juristen/AnwÃ¤lte/Gericht','Personalvermittlung/-beratung','Personalwesen/-entwicklung','QualitÃ¤tsmanagement','Regulatory Affairs','Sachbearbeitung/SprachÃ¼bersetzung','Sekretariat/Empfang', 'None'],
    "Banking/Versicherungswesen": ["Mustang"'Aktuariat/Mathematik','Anlageberatung/Private Banking','Asset/Portfolio Management','Back-Office/Zahlungsverkehr','Finanz-/Business Analyse','Fonds/Wertschriften/Handel','Kredit/Hypotheken','Kundenberatung/Relationship Mgmt','Projekt Management','Risk Mgmt/Compliance','Treasury/Controlling/Audit','Underwriting/Product Mgmt','Versicherung: Agenturen','Versicherung: Akquisition/Beratung','Versicherung: Schaden/Leistungen', 'None'],
    "Bau/Architektur/Engineering": ['Architektur/Bauplanung/Zeichner','Bauleitung/Bauingenieure/AVOR','Baupersonal/GerÃ¼stbau/Maschinen','Bauspengler/Dachdecker','Boden-/Plattenleger','Elektro Planung/Montage','Gartenbau/Landschaftsarchitektur','Heizung/LÃ¼ftung/Klima Planung/Montage','Holzbau/Zimmermann','Innenarchitektur/Dekoration','Maler/Gipser','Maurer','Metall/Fassadenbau','SanitÃ¤r Planung/Montage', 'None'],
    "Bewachung/Polizei/Zoll/Rettung": ['Bewachung/Kontrolle/Sicherheit','Polizei/MilitÃ¤r','Rettung SanitÃ¤t/Feuerwehr','Zoll/Grenzkontrolle', 'None'],
    "Chemie/Pharma/Biotechnologie": ['Apotheken/Drogerien','Biologie/Biotechnologie','Chemie F+E/Analytik/Produktion','Pharma F+E/Analytik/Produktion','QualitÃ¤tswesen/-management', 'None'],
    "Einkauf/Logistik/Trading": ['Einkauf/Procurement','Import/Export/Trading','Logistik/Supply Chain', 'None'],
    "Elektronik/Technik/Uhren": ['Elektronik/Elektrotechnik','Energie-/GebÃ¤ude-/Haustechnik','Kundendienst/Beratung','Mechanik/Feinmechanik/Uhren','Medizinaltechnik','QualitÃ¤tswesen/-management','Servicetechnik','Werkzeugbau', 'None'],
    "Fahrzeuge/Handwerk/Lager/Transport": ['Carosseriespengler/Lackierer','Facility Management/Reinigung','Fahrzeugmechanik/-diagnostik','Holzhandwerk','Lager','Land-/Forstwirtschaft','Metallhandwerk','Personentransport Flug/See/Zug/Strasse','Textil/Schneider', 'None'],
    "Finanzen/Treuhand/Immobilien": ['Audit/Revision/WirtschaftsprÃ¼fung','Buchhaltung','Controlling','Finanz-/Rechnungswesen','Immobilien Handel/Bewirtschaftung','Inkasso','Lohnbuchhaltung','Steuern','Treuhand', 'None'],
    "Gastronomie/Lebensmittel/Tourismus": ['BetriebsfÃ¼hrung/-administration','KÃ¼che','Lebensmitteltechnologie/-ingenieure','Metzger/BÃ¤cker/Konditor/KÃ¤ser','RÃ©ception/Portier/Concierge','Reiseberatung/Reiseleitung','Service/Bar','Zimmer/Lingerie/Hauswirtschaft', 'None'],
    "Grafik/Typografie/Druck": ['Drucker','Grafik/Design','Typografie/Polygrafie', 'None'],
    "Informatik/Telekommunikation": ['Beratung/Wirtschaftsinformatik','Datenbank Spezialisten/Entwicklung','ERP/SAP/CRM','Netzwerk Spezialisten/Engineers','Projekt Management/Analyse','Software Architektur/Engineering','Software Programmierung','System Administration','System Engineering','Testing/Audit/Security','User Help Desk/Support/Training','Web Programmierung/Mobile','Web Publishing/Design', 'None'],
    "Marketing/Kommunikation/Redaktion": ['Marketing/Direktmarketing','Medien/Redaktion/Verlagswesen','Online Marketing/Social Media','PR/Corporate Communications','Produkt-/Brand Management','Werbeberatung/-kommunikation', 'None'],
    "Maschinen-/Anlagenbau/Produktion": ['Anlagen-/Apparatebau','Automation/Verfahrenstechnik','CNC-Mechanik/Polymechanik','Konstruktion','Maschinenbau','Produktion/Fertigung/Montage','QualitÃ¤tswesen/-management','Servicetechnik/Wartung', 'None'],
    "Medizin/Pflege/Therapie": ['Ã„rzte - Assistenz-/Fach-/OberÃ¤rzte','Akutpflege','Hebammen','Labor/Analyse','Langzeitpflege','Med. Praxisassistenz/Sekretariat','Psychologie/Psychiatrie','Radiologie','Therapie/OrthopÃ¤die','Tiermedizin', 'None'],
    "Sport/Wellness/Kultur": ['Coiffeur/Kosmetik','Kultur/Unterhaltung','Massage/Sport/Wellness','Mode/Film/Theater/Radio','MuseumKunst/Kunsthandwerk', 'None'],
    "Verkauf/Kundendienst/Innendienst": ['Category Management','Kundenberatung/Key Account Mgmt','Kundendienst/Call Center','Telefonverkauf/Telesale','Verkauf Aussendienst/Spezialisten','Verkauf Beratung/Management','Verkauf Detailhandel','Verkauf Innendienst/Back-Office', 'None'],
    "Verwaltung/Bildung/Soziales": ['Lehrer/Dozenten','NGO/NPO/Hilfswerke','Oeffentliche Verwaltung','Sozial-/HeilpÃ¤dagogen','Sozial-/Jugendarbeit','Wissenschaft/Forschung', 'None']
}

user_input = ''


input_message = "Pick an option:\n"

for index, item in enumerate(options):
    input_message += f'{index+1}) {item}\n'

input_message += 'Your choice: '

while user_input not in options:
    user_input = input(input_message)
print('You picked: ' + user_input)



Pick an option:
1) Administration/HR/Consulting/CEO
2) Banking/Versicherungswesen
3) Bau/Architektur/Engineering
4) Bewachung/Polizei/Zoll/Rettung
5) Chemie/Pharma/Biotechnologie
6) Einkauf/Logistik/Trading
7) Elektronik/Technik/Uhren
8) Fahrzeuge/Handwerk/Lager/Transport
9) Finanzen/Treuhand/Immobilien
10) Gastronomie/Lebensmittel/Tourismus
11) Grafik/Typografie/Druck
12) Informatik/Telekommunikation
13) Marketing/Kommunikation/Redaktion
14) Maschinen-/Anlagenbau/Produktion
15) Medizin/Pflege/Therapie
16) Sport/Wellness/Kultur
17) Verkauf/Kundendienst/Innendienst
18) Verwaltung/Bildung/Soziales
Your choice: Administration/HR/Consulting/CEO
You picked: Administration/HR/Consulting/CEO


In [5]:
#print(options[user_input])
#print(options["brand"])
#print(input_message_2) 
user_input_2 = ''
input_message_2 = "Pick an option:\n"

for index, item in enumerate(options[user_input]):
    input_message_2 += f'{index+1}) {item}\n'
    
input_message_2 += 'Your choice: '

while user_input_2 not in options[user_input]:
    user_input_2 = input(input_message_2)

Pick an option:
1) Beratung/Unternehmensentwicklung
2) CEO/GeschÃ¤ftsfÃ¼hrung
3) Direktionsassistenz
4) Juristen/AnwÃ¤lte/Gericht
5) Personalvermittlung/-beratung
6) Personalwesen/-entwicklung
7) QualitÃ¤tsmanagement
8) Regulatory Affairs
9) Sachbearbeitung/SprachÃ¼bersetzung
10) Sekretariat/Empfang
11) None
Your choice: Per  sonalwesen/-entwicklung
Pick an option:
1) Beratung/Unternehmensentwicklung
2) CEO/GeschÃ¤ftsfÃ¼hrung
3) Direktionsassistenz
4) Juristen/AnwÃ¤lte/Gericht
5) Personalvermittlung/-beratung
6) Personalwesen/-entwicklung
7) QualitÃ¤tsmanagement
8) Regulatory Affairs
9) Sachbearbeitung/SprachÃ¼bersetzung
10) Sekretariat/Empfang
11) None
Your choice: Personalwesen/-entwicklung


In [6]:
print(user_input)
print(user_input_2)

Administration/HR/Consulting/CEO
Personalwesen/-entwicklung


In [7]:
data = data[data.lvl1_jobcategory_name == user_input]
data = data[data.jobcategory_name == user_input_2]
data = data.drop('Unnamed: 0.1',axis=1)
data = data.drop('Unnamed: 0',axis=1)
print(data)

                  ad_id active_start_on active_end_on  \
154      jobsse-5851136      2014-06-03    2014-06-13   
214      jobsse-5757031      2014-04-09    2014-05-01   
230      jobsse-8554204      2018-11-19    2018-11-23   
288      jobsse-8768522      2019-02-28    2019-03-01   
313      jobsse-9405060      2019-12-31    2020-01-13   
...                 ...             ...           ...   
132647   jobsse-6801005      2016-02-16    2016-03-10   
132650   jobsse-8087715      2018-03-22    2018-04-01   
132702   jobsse-7550223      2017-05-19    2017-06-02   
132713   jobsse-6091093      2014-10-27    2014-11-06   
132747  jobsse-10651190      2021-09-09    2021-09-16   

                                                    title  \
154                 Head Human Resources Management (m/w)   
214     Spezialist/-in Personal- und Organisationsentw...   
230                      HRBP/HR Manager Operations (m/w)   
288                        Fachperson Personalentwicklung   
313       

In [8]:
# Vergleichen unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()
count = 0
while year <= data['year'].max():
    wordsets_phrases = data.loc[data['year'] == year]['Text_Anforderungen'] # Text Detok, da n-grams nach dem gesucht wurden
    for word in set_phrases['word']:
        count = sum(1 for s in wordsets_phrases if word in s)
        index = set_phrases[set_phrases['word']==word].index
        set_phrases.loc[index, year] = count
        set_phrases.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))    
    
    year += 1

In [9]:
set_fix = pd.read_excel("Fixwoerter.xlsx")

In [10]:
# Anforderungen Tokenisieren
set_fix['text_tok'] = set_fix.apply(lambda row: nltk.word_tokenize(row['word']), axis=1)
set_fix['text_tok'].head(10)

0      [abap]
1         [c]
2      [java]
3     [plsql]
4    [python]
5         [r]
6    [apache]
7     [azure]
8    [citrix]
9      [hana]
Name: text_tok, dtype: object

In [11]:
# Anforderungen in einem Set speichern
new_set = set()
set_fix['text_tok'].apply(new_set.update)
u = new_set

In [12]:
# Vergleiche unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()

while year <= data['year'].max():    
    wordsets_fix = [ frozenset(document.split(' ')) for document in data.loc[data['year'] == year]['Text_Anforderungen'] ]
    for word in u:
        count = sum( 1 for s in wordsets_fix if word in s )
        index = set_fix[set_fix['word']==word].index
        set_fix.loc[index, year] = count
        set_fix.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))

    year += 1

In [13]:
anforderungen = set_phrases.append(set_fix, ignore_index=True)

C:\Users\Sven Konermann\AppData\Local\Temp\ipykernel_25216\2611590460.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anforderungen = set_phrases.append(set_fix, ignore_index=True)


In [14]:
# Anforderungen mit den meisten Nennungen
anforderungen.sort_values(by=2021,ascending=False).head(5)

,word,kategorie,kategorie_name,hauptkategorie,2014,2014 %,2015,2015 %,2016,2016 %,...,2017 %,2018,2018 %,2019,2019 %,2020,2020 %,2021,2021 %,text_tok
10,deutsch,8,Sprachkenntnisse,Sprachen,436.0,0.800000,340.0,0.748899,291.0,0.729323,...,0.724289,323.0,0.725843,275.0,0.708763,321.0,0.812658,229.0,0.741100,NaN
21,teamfähig,12,Soft Skills,Soft Skills,169.0,0.310092,152.0,0.334802,124.0,0.310777,...,0.253829,147.0,0.330337,123.0,0.317010,145.0,0.367089,115.0,0.372168,NaN
68,berufserfahrung,9,Berufserfahrung,Abschlüsse und Berufserfahrung,241.0,0.779935,203.0,0.656958,160.0,0.517799,...,0.579288,175.0,0.566343,167.0,0.540453,184.0,0.595469,114.0,0.368932,[berufserfahrung]
11,englisch,8,Sprachkenntnisse,Sprachen,282.0,0.517431,229.0,0.504405,201.0,0.503759,...,0.407002,201.0,0.451685,157.0,0.404639,160.0,0.405063,111.0,0.359223,NaN
25,selbständig,12,Soft Skills,Soft Skills,210.0,0.385321,165.0,0.363436,145.0,0.363409,...,0.308534,149.0,0.334831,128.0,0.329897,131.0,0.331646,96.0,0.310680,NaN


In [16]:
# Nur für Daten mit Jobcloud
anforderungen.to_csv("ausgewertete_Anforderungen.csv", sep=";")